### Laboratorio 6 - Generative Adversarial Network
Stefano Aragoni, Carol Arévalo

-----------

En esta práctica se diseñó una Generative Adversarial Network (GAN) con el propósito de poder generar imágenes artificiales que imiten la distribución de los datos originales Para esto, fue necesario diseñar una red neuronal que fuera capaz de generar imágenes, y otra red neuronal que fuera capaz de diferenciar entre imágenes reales y generadas. 

A continuación se muestra el código utilizado para la creación de la GAN, así como los resultados obtenidos.

------- 

##### Importar librerías

Como primer paso, se importaron las librerías necesarias para la creación de la GAN.

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten, Input
import tensorflow as tf
from keras.layers import Dense, Reshape, Dropout, LeakyReLU, Flatten, BatchNormalization, Conv2D, Conv2DTranspose
from keras.models import Sequential
from keras.optimizers import Adam
import os
import cv2
from tqdm import tqdm 
import pickle
from keras.preprocessing.image import ImageDataGenerator

--------
### **Preparación de Datos**

##### Cargar el dataset de CelebA y Preprocesamiento de Datos 

Para iniciar, se descargó el dataset de CelebA. Este conjunto de datos consta de más de 200,000 imágenes a color, de 128 X 128 X 3 c/u. A continuación se muestra la ubicación de las imágenes en el dataset.

In [34]:
# Direcciones de los archivos
fotos_dir = 'archive/img_align_celeba/'
fotos_dir_class = 'archive/img_align_celeba/img_align_celeba/'

# Cantidad de fotos en el directorio
n_fotos = len(os.listdir(fotos_dir_class))
print(n_fotos, "fotos en el directorio")

202599 fotos en el directorio


A través de la librería de Keras, se cargaron las imágenes por batches.

Asimismo, se les aplicó un preprocesamiento, el cual consistió en normalizar los valores de los pixeles de las imágenes, recortarlas y redimensionarlas a 64 X 64 X 3.

In [25]:
# Batch size
batch_size = 32

# Tamaño de las imágenes
img_size = 64

In [36]:
# Leer las imagenes -> Función recomendada por Prof. Luis Furlan.

datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,                    # PRE-PROCESAMIENTO: Normalizar los valores de los pixeles
    rotation_range=20,
    width_shift_range=0.2,  
    height_shift_range=0.2,
    horizontal_flip=True,
)

In [37]:
# Carga las imagenes de entrenamiento

train_generator = datagen.flow_from_directory(
    fotos_dir,
    target_size=(img_size, img_size),        # PRE-PROCESAMIENTO: RECORTAR Y REDIMENSIONAR IMÁGENES
    batch_size=batch_size,
    class_mode=None,
    subset='training'
)

Found 202599 images belonging to 1 classes.


--------
### **Implementación de la GAN**

##### Diseño del generador y el discriminador

A continuación se demuestra el modelo del <font color=orange>generador</font>.

In [62]:
generador = Sequential()
generador.add(Dense(8 * 8 * 256, input_shape=[tamanio_codificacion])) 
generador.add(Reshape([8, 8, 256]))
generador.add(BatchNormalization())
generador.add(Conv2DTranspose(128, kernel_size=5, strides=2, padding="same", activation="relu"))  
generador.add(BatchNormalization())
generador.add(Conv2DTranspose(64, kernel_size=5, strides=2, padding="same", activation="relu"))  
generador.add(BatchNormalization())
generador.add(Conv2DTranspose(3, kernel_size=5, strides=2, padding="same", activation="sigmoid")) 

A continuación se demuestra el modelo del <font color=orange>discriminador</font>.

In [63]:
discriminador = Sequential()
discriminador.add(Conv2D(64, kernel_size=5, strides=2, padding="same", input_shape=(64, 64, 3)))
discriminador.add(LeakyReLU(0.3))
discriminador.add(Dropout(0.5))
discriminador.add(Conv2D(128, kernel_size=5, strides=2, padding="same"))
discriminador.add(LeakyReLU(0.3))
discriminador.add(Dropout(0.5))
discriminador.add(Flatten())
discriminador.add(Dense(1, activation="sigmoid"))

##### Definición de funciones de pérdida y optimizadores

Con los modelos listos, se procedió a definir las funciones de pérdida y los optimizadores. Esto con el propósito de poder entrenar la GAN. Más específicamente, se utilizó la función de <font color=orange>Binary Cross Entropy</font> (BCE) como función de pérdida, y el <font color=orange>optimizador Adam</font>. 

Asimismo, se indicó que el discriminador no se entrenaría durante el entrenamiento de la GAN, ya que el objetivo es entrenar al generador para que engañe al discriminador.

In [64]:
GAN = Sequential([generador, discriminador])

discriminador.compile(loss="binary_crossentropy", optimizer="adam")
discriminador.trainable = False

GAN.compile(loss = "binary_crossentropy", optimizer = "adam")

--------
### **Entrenamiento de la GAN**

##### Implementación del bucle de entrenamiento

In [65]:
# Si se desea que el entrenamiento sea más
#   rápido, se puede tomar un valor mayor
tamanio_tanda = 32

# mis_datos = X_entreno
mis_datos = train_generator[0]

datos = tf.data.Dataset.from_tensor_slices(mis_datos).shuffle(buffer_size = 1000)

datos = datos.batch(tamanio_tanda, 
                    drop_remainder = True).prefetch(1)

epocas = 20


# Tomar los componentes por separado
generador, discriminador = GAN.layers

In [ ]:
# ...

# Number of epochs for training
epochs = 20

# Generator and discriminator components
generator, discriminator = GAN.layers

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}")
    i = 0
    
    # Iterate through batches in the training generator
    for X_batch in train_generator:
        i += 1
        if i % 25 == 0:
            print(f"\tBatch {i} of {len(train_generator)}")
            
        # Fase 1 - Entrenamiento del DISCRIMINADOR
        
        # Create noise
        noise = tf.random.normal(shape=[batch_size, tamanio_codificacion])
        
        # Generate fake images based on noise
        generated_images = generator(noise)
        
        # Concatenate fake and real images
        X_fake_vs_real = tf.concat([generated_images, X_batch], axis=0)
        
        # Set targets: 0 for fake images, 1 for real images
        y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
        
        # Allow the discriminator to be trainable
        discriminator.trainable = True
        
        # Train the discriminator on this batch
        discriminator.train_on_batch(X_fake_vs_real, y1)
        
        # Fase 2 - Entrenamiento del GENERADOR
        
        # Create some noise
        noise = tf.random.normal(shape=[batch_size, tamanio_codificacion])
        
        # Make the discriminator believe that the fake images are real
        y2 = tf.constant([[1.]] * batch_size)
        
        # Prevent a warning
        discriminator.trainable = False
        
        # Train the GAN (generator) on this batch
        GAN.train_on_batch(noise, y2)

print("Training Completed")


Epoch 1


InvalidArgumentError: {{function_node __wrapped__ConcatV2_N_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} ConcatOp : Dimension 1 in both shapes must be equal: shape[0] = [32,28,28,1] vs. shape[1] = [32,64,64,3] [Op:ConcatV2] name: concat

In [ ]:
ruido = tf.random.normal(shape = [10, tamanio_codificacion])

imagenes = generador(ruido)

In [ ]:
plt.imshow(imagenes[0])

##### Visualización de los resultados durante el entrenamiento

--------
### **Reflexión**

Reflexione sobre lo aprendido en la sesión teórica y cómo se aplicó en el laboratorio. Algunos 
puntos que podrían considerar en su reflexión incluyen:

1. ¿Qué conceptos de la teoría encontraron más desafiantes y por qué?


2. ¿Cómo les ayudó el laboratorio a consolidar o entender mejor estos conceptos?


3. ¿Qué aplicaciones potenciales ven para las GANs en la industria o en la investigación?

4. ¿Qué limitaciones o preocupaciones éticas pueden identificar en el uso de GANs?


5. ¿Cómo se sienten con respecto a la implementación y entrenamiento de GANs después de la  experiencia práctica?